In [48]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import numpy as np
from scipy.optimize import minimize

In [49]:
db_path = '../data/processed/database_final.db'

if not os.path.isfile(db_path):
    raise FileNotFoundError(f"Database file not found at {db_path}")

conn = sqlite3.connect(db_path)

In [50]:
df_stock = pd.read_sql_query("SELECT * from STOCKEX", conn)
df_gold = pd.read_sql_query("SELECT * from GOLD", conn)
df_bond = pd.read_sql_query("SELECT * from BOND", conn)
df_hpi = pd.read_sql_query("SELECT * from HPI", conn)
df_btc = pd.read_sql_query("SELECT * from BTC", conn)
df_rf = pd.read_sql_query("SELECT * from RF", conn)

# Markowitz Portfolio:

In [51]:
stock_ret = df_stock["real_return_stockex"]
gold_ret = df_gold["real_return_gold"]
bond_ret = df_bond["real_return_bond"]
hpi_ret = df_hpi["real_return_hpi"]
btc_ret = df_btc["real_return_bitcoin"]

# Risk Free Rate:

In [52]:
df_rf["real_return_riskfree"].head()

0   -0.303951
1   -0.314440
2   -0.350748
3   -0.390957
4   -0.402797
Name: real_return_riskfree, dtype: float64

In [53]:
risk_free1 = df_rf["real_return_riskfree"].median()
print("The median real return of US Gov Bond(10Y) Yields:", risk_free1)

The median real return of US Gov Bond(10Y) Yields: -0.13708159972828393


In [54]:
risk_free2 = df_rf["real_return_riskfree"].mean()
print("The mean real return of US Gov Bond(10Y) Yields:", risk_free2)

The mean real return of US Gov Bond(10Y) Yields: -0.20866998432423317


# Portfolio with Median Risk Free Rate:

In [55]:
returns = pd.concat([stock_ret, gold_ret, bond_ret, hpi_ret, btc_ret], axis=1)
returns.columns = ['Stock', 'Gold', 'Bond', 'HPI', 'Bitcoin']

mean_returns = returns.mean()
cov_matrix = returns.cov()

num_assets = len(mean_returns)

def portfolio_performance(weights, mean_returns, cov_matrix):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate= risk_free1):
    p_return, p_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    sharpe_ratio = (p_return - risk_free_rate) / p_volatility
    return -sharpe_ratio  

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 
bounds = tuple((0, 1) for _ in range(num_assets))  

initial_guess = num_assets * [1. / num_assets, ]

result = minimize(negative_sharpe_ratio, initial_guess,
                  args=(mean_returns, cov_matrix),
                  method='SLSQP', bounds=bounds, constraints=constraints)

optimal_weights = result.x

optimal_return, optimal_volatility = portfolio_performance(optimal_weights, mean_returns, cov_matrix)

weights_percentages = optimal_weights * 100

print("Optimal Portfolio Weights (%):")
for asset, weight in zip(returns.columns, weights_percentages):
    print(f"{asset}: {weight:.2f}%")

print(f"\nExpected Portfolio Return: {optimal_return:.2%}")
print(f"Expected Portfolio Volatility: {optimal_volatility:.2%}")

Optimal Portfolio Weights (%):
Stock: 7.71%
Gold: 24.67%
Bond: 0.00%
HPI: 67.15%
Bitcoin: 0.47%

Expected Portfolio Return: 22.05%
Expected Portfolio Volatility: 41.25%


# Portfolio with Mean Risk Free Rate:

In [56]:
returns = pd.concat([stock_ret, gold_ret, bond_ret, hpi_ret, btc_ret], axis=1)
returns.columns = ['Stock', 'Gold', 'Bond', 'HPI', 'Bitcoin']

mean_returns = returns.mean()
cov_matrix = returns.cov()

num_assets = len(mean_returns)

def portfolio_performance(weights, mean_returns, cov_matrix):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate= risk_free2):
    p_return, p_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    sharpe_ratio = (p_return - risk_free_rate) / p_volatility
    return -sharpe_ratio  

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 
bounds = tuple((0, 1) for _ in range(num_assets))  

initial_guess = num_assets * [1. / num_assets, ]

result = minimize(negative_sharpe_ratio, initial_guess,
                  args=(mean_returns, cov_matrix),
                  method='SLSQP', bounds=bounds, constraints=constraints)

optimal_weights = result.x

optimal_return, optimal_volatility = portfolio_performance(optimal_weights, mean_returns, cov_matrix)

weights_percentages = optimal_weights * 100

print("Optimal Portfolio Weights (%):")
for asset, weight in zip(returns.columns, weights_percentages):
    print(f"{asset}: {weight:.2f}%")

print(f"\nExpected Portfolio Return: {optimal_return:.2%}")
print(f"Expected Portfolio Volatility: {optimal_volatility:.2%}")

Optimal Portfolio Weights (%):
Stock: 3.48%
Gold: 8.40%
Bond: 57.76%
HPI: 30.21%
Bitcoin: 0.15%

Expected Portfolio Return: 1.34%
Expected Portfolio Volatility: 19.77%
